In [0]:
df = spark.read.load("/FileStore/tables/Order.csv",format='csv')    # reading file
#display(df)  (display is a databricks specific function)
# df.show() #show
df = spark.read.load("/FileStore/tables/Order.csv",format='csv',header=True) #treates first row as a header
# display(df)
df.printSchema()
df1 = spark.read.load("/FileStore/tables/Order.csv",header=True,format='csv',inferschema=True) #adjust the schema by reading data in file
df1.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- SalesChannel: string (nullable = true)
 |-- OrderPriority: string (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- UnitsSold: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- UnitCost: string (nullable = true)
 |-- TotalRevenue: string (nullable = true)
 |-- TotalCost: string (nullable = true)
 |-- TotalProfit: string (nullable = true)

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- SalesChannel: string (nullable = true)
 |-- OrderPriority: string (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- UnitsSold: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- UnitCost: double (nullable = true)
 |-- TotalRevenue: double (nullable = true)
 |-- TotalCost: double (nullable = true)
 |-- TotalProfit: double (nullable = tru

In [0]:
#to define own schema 2 approches using traditional pyspark or using sql
#using traditional method
from pyspark.sql.types import *
orderschema = StructType([StructField("Region", StringType(), True),
                         StructField("Country", StringType(), True)])
df = spark.read.load("/FileStore/tables/Order.csv",header=True,format='csv', schema=orderschema) #schema will take our own schema
# display(df)
#sql method (most using and easy way)
orderschema_sql = "region string, country String, ItemType String, SalesChannel String, OrderPriority String"
df_main = spark.read.load("/FileStore/tables/Order.csv",header=True,format='csv', schema=orderschema_sql) #schema will take our own schema
df_main.show(10)
df_main.printSchema()
df2= df1.drop("OrderPriority","ItemType")  # drop helps us to remove columns from our data frame
df2.show(10)


+--------------------+----------+----------+------------+-------------+
|              region|   country|  ItemType|SalesChannel|OrderPriority|
+--------------------+----------+----------+------------+-------------+
|Middle East and N...|     Libya| Cosmetics|     Offline|            M|
|       North America|    Canada|Vegetables|      Online|            M|
|Middle East and N...|     Libya| Baby Food|     Offline|            C|
|                Asia|     Japan|    Cereal|     Offline|            C|
|  Sub-Saharan Africa|      Chad|    Fruits|     Offline|            H|
|              Europe|   Armenia|    Cereal|      Online|            H|
|  Sub-Saharan Africa|   Eritrea|    Cereal|      Online|            H|
|              Europe|Montenegro|   Clothes|     Offline|            M|
|Central America a...|   Jamaica|Vegetables|      Online|            H|
|Australia and Oce...|      Fiji|Vegetables|     Offline|            H|
+--------------------+----------+----------+------------+-------

In [0]:
#creating data frame manually
# we need columns(schema) and rows for creating data frame
# from pyspark.sql import *
from pyspark.sql.functions import *
row1 = Row("Pavan", 1, 50000)
row2 = Row("Sai", 2, 75000)
employee_schema = ("name String, employee_id Integer, employee_salary Integer")
df = spark.createDataFrame([row1, row2], schema=employee_schema)
display(df)

#for selecting particular columns 5 methods 
df1 = df.select("name", "employee_id") #for selecting particular multiple columns (popularly used)
df2 = df.select(col("name")) #drawback -- col,column,expr only can select 1 argument
df3 = df.select(column("name"))
df4 = df.select(expr("name")) #popularly used
df5 = df.select(df.name,df.employee_salary)
df6 = df.selectExpr("name as full_name", "employee_id", "employee_salary") #for column selection along with column name change
display(df6)
# display(df4)


name,employee_id,employee_salary
Pavan,1,50000
Sai,2,75000


full_name,employee_id,employee_salary
Pavan,1,50000
Sai,2,75000


In [0]:
#
# display(df_main)
df1 = df_main.withColumn("Flag", lit(123))  #literals are constant values 
df2 = df1.withColumn("Country_india", expr("country == 'India'"))  #for inserting new column
# display(df2)
df3 = df2.withColumnRenamed("Country","ALL_Countries") # for renaming existing column
# display(df3)
df4 = df2.selectExpr("Country as countries_all", "region", "SalesChannel") #for column selection along with column name change
# display(df4)
df5 = df4.drop("region")  #dropping particular columns
############################################################################################

df6 = df1.where("Region = 'Asia' ") #to filter records can use either where or Filter 
df7 = df1.select("Region","Country").distinct() #for unique records selection
df7.show(10)


+--------------------+----------+
|              Region|   Country|
+--------------------+----------+
|                Asia|     Japan|
|Middle East and N...|     Libya|
|  Sub-Saharan Africa|   Eritrea|
|  Sub-Saharan Africa|      Chad|
|              Europe|Montenegro|
|       North America|    Canada|
|Australia and Oce...|      Fiji|
|              Europe|    Greece|
|Central America a...|   Jamaica|
|              Europe|   Armenia|
+--------------------+----------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import *
#union in spark
df1 = spark.read.load("/FileStore/tables/Order.csv",header=True,format='csv')
df2 = df1.select("Region","Country","OrderId")
df3 = df2.filter("Country == 'Libya'")
df4 = df2.filter("Country == 'Japan'")
df4_union = df3.union(df4) #accepts only one argument merging of data of same column
display(df4_union)
df4_union.count()

#orderby -- sorting
df4_union.sort(col("Region").desc()).show(20, truncate=False, vertical = True) #displays in vertical format(20 rows), truncation not allowed
df5 = df4_union.sort(col("Region").asc()) #sorting in ascending order
df6 = df5.limit(10) #limiting the columns to 10
display(df6)

df6.collect() #not recommended because it collects all data from all machines and then prints the data
df7 = df6.describe() #describes mean,count,median
df7.show()

Region,Country,OrderId
Middle East and North Africa,Libya,686800706
Middle East and North Africa,Libya,246222341
Middle East and North Africa,Libya,964214932
Middle East and North Africa,Libya,635122907
Middle East and North Africa,Libya,276225316
Middle East and North Africa,Libya,854095017
Asia,Japan,161442649
Asia,Japan,981086671
Asia,Japan,823699796
Asia,Japan,629925000


-RECORD 0-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 686800706                    
-RECORD 1-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 246222341                    
-RECORD 2-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 964214932                    
-RECORD 3-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 635122907                    
-RECORD 4-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 276225316                    
-RECORD 5-------------------------------
 Region  | Middle East and North Africa 
 Country | Libya                        
 OrderId | 854095017                    
-RECORD 6-------

Region,Country,OrderId
Asia,Japan,161442649
Asia,Japan,981086671
Asia,Japan,823699796
Asia,Japan,629925000
Asia,Japan,107172334
Asia,Japan,885129249
Asia,Japan,551725089
Asia,Japan,453569972
Middle East and North Africa,Libya,686800706
Middle East and North Africa,Libya,246222341


+-------+--------------------+-------+-------------------+
|summary|              Region|Country|            OrderId|
+-------+--------------------+-------+-------------------+
|  count|                  10|     10|                 10|
|   mean|                null|   null|      5.526773807E8|
| stddev|                null|   null|3.067292352880238E8|
|    min|                Asia|  Japan|          107172334|
|    max|Middle East and N...|  Libya|          981086671|
+-------+--------------------+-------+-------------------+



In [0]:
#string manipulation functions
df7 = df6.select(upper(col("Region"))).show(2, truncate=False) #all capital letters
df8 = df6.select(lower(col("Region"))).show(2) # all small letters
df6.select(initcap(col("Region"))).show(2) #first letter capital
df6.select(trim(col("Region"))).show(truncate=False) #removing spaces can use ltrim or rtrim for right or left space removal
df6.select(lpad(col("Region"),20,'+')).show(truncate=False) #can do for right padding also
df6.select(concat(col("Region"),col("Country"))).show()  #concating within same data frame
df6.select(concat_ws('--', col("Region"),col("Country"))).show(truncate=False) #concating with separator
###################################################################################


+-------------+
|upper(Region)|
+-------------+
|ASIA         |
|ASIA         |
+-------------+
only showing top 2 rows

+-------------+
|lower(Region)|
+-------------+
|         asia|
|         asia|
+-------------+
only showing top 2 rows

+---------------+
|initcap(Region)|
+---------------+
|           Asia|
|           Asia|
+---------------+
only showing top 2 rows

+----------------------------+
|trim(Region)                |
+----------------------------+
|Asia                        |
|Asia                        |
|Asia                        |
|Asia                        |
|Asia                        |
|Asia                        |
|Asia                        |
|Asia                        |
|Middle East and North Africa|
|Middle East and North Africa|
+----------------------------+

+--------------------+
|lpad(Region, 20, +) |
+--------------------+
|++++++++++++++++Asia|
|++++++++++++++++Asia|
|++++++++++++++++Asia|
|++++++++++++++++Asia|
|++++++++++++++++Asia|
|+++++

In [0]:
from pyspark.sql.functions import *
#date manipulation functions
#range describes how many rows should be in new_data frame
date_df = spark.range(10).withColumn("Date_current",current_date()).withColumn("Time_stamp",current_timestamp())
date_df.show(truncate=False)
date_df.select(date_add("Date_current", 5), date_sub("Date_current", 5)).show()  #date addition or subtraction by constant

#converting to date data type
const_df = spark.range(2).withColumn("Date",lit("2024-05-20"))
const_df.show()
const_df.printSchema()
updated_df = const_df.select(to_date(col("Date")).alias("new_date")) #to_date converts string to date format,data should be date format 
updated_df.printSchema()
updated_df.select(date_add(col("new_date"),5).alias("new_date")).show()
updated_df.select(month(col("new_date"))).show() #year,month,day can be extracted
updated_df.select(dayofmonth(col("new_date"))).show() 
#can also extract the minute,hour, second if it is a time stamp format

+---+------------+-----------------------+
|id |Date_current|Time_stamp             |
+---+------------+-----------------------+
|0  |2024-10-17  |2024-10-17 02:56:34.765|
|1  |2024-10-17  |2024-10-17 02:56:34.765|
|2  |2024-10-17  |2024-10-17 02:56:34.765|
|3  |2024-10-17  |2024-10-17 02:56:34.765|
|4  |2024-10-17  |2024-10-17 02:56:34.765|
|5  |2024-10-17  |2024-10-17 02:56:34.765|
|6  |2024-10-17  |2024-10-17 02:56:34.765|
|7  |2024-10-17  |2024-10-17 02:56:34.765|
|8  |2024-10-17  |2024-10-17 02:56:34.765|
|9  |2024-10-17  |2024-10-17 02:56:34.765|
+---+------------+-----------------------+

+-------------------------+-------------------------+
|date_add(Date_current, 5)|date_sub(Date_current, 5)|
+-------------------------+-------------------------+
|               2024-10-22|               2024-10-12|
|               2024-10-22|               2024-10-12|
|               2024-10-22|               2024-10-12|
|               2024-10-22|               2024-10-12|
|               202